$$ TASK-3$$

### 1. Introduction

**Classical Convolution**

**Definition :** In the context of Convolutional Neural Networks, Classical Convolution denotes a technique where a small filter or kernel slides or convolves over parts of the input data to extract features. All in all, the procedure entails the sliding of a filter across an input image and the performance of element-wise multiplication and summation.

**Operation :** A CNN processes small regions of an image at a time because of its convolutional layer, enabling the model to learn local patterns such as edges and textures. This convolves each region with a kernel to give one output pixel, typically forming a larger feature map.

**Feature Extraction :** Further feature extraction or classification is performed by additional layers after processing the resulting feature maps. The ability to understand spatial hierarchies in images is the main strength of CNNs because they stack multiple convolutional layers.


**Quantum Convolution**

**Definition :** Quantum convolution can be defined as the generalization of classical convolution into Quantum Variational Circuits. In quantum convolution local patches from the input data will be embedded in the quantum circuit and after that, quantum operations are applied over there.

**Process :**

**Circuit Embedding :** A small portion of the input image, e.g., a 2x2 patch of pixels, is physically embedded in the circuit. The parametrized quantum gates represent a mapping from classical data to quantum states.

**Quantum Computation :** The unitary transformation application  Quantum computation on the encoded data can be shown to be effectively given by the application.

**Measurement :** Read out the quantum circuit in order to end up with classical expectation values associated with the probability of the state measurement which would be extracted quantum features related to the input data.

**Output Mapping :** Each expectation value corresponds to a different O pixel channel; thus, the need to perform a classical layer, similar to the convolutional case.

**Iterate :** Do this for every patch of the image to obtain a new image-like object that can be further processed.

**Benefits :** Quantum circuits can potentially create complex transformations that are computationally expensive or intractable for classical systems. They have the ability to capture intricate data patterns through entanglement and superposition.


### 2. General Setup

Here, we set up the Python environment and all the necessary libraries to conduct the experiment.

**A) PennyLane :** PennyLane is a quantum machine learning library by which one can easily do quantum or classical computation. We are using it for quantum circuit simulations.

**B) TensorFlow :** A popular machine learning library. We'll be using it here to build the classical components of our model and to train the whole network.

**C) Matplotlib :** An extensive plotting library. We will use it here to display our dataset and some plots from the results.



In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

**2.1 Hyper-Parameters :**

**A) n_epochs :** The number of epochs that we would like to train our classical model. 

**B) n_layers :** Number of the random layers used in our quantum circuit.

**C) n_train :** Number of samples for training.

**D) n_test :** Number of samples for testing.

**E) SAVE_PATH :** Save path of preprocessing the images

**F) PREPROCESS :** Boolean flag to decide for quantum preprocessing or not

**G) Random Seeds :** Fixing the seeds to make results reproducible


In [ ]:
n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 50    # Size of the train dataset
n_test = 30     # Size of the test dataset

SAVE_PATH = "Copy path of new folder in which mnist.npz is located"  # Data saving folder
PREPROCESS = True           # If False, skip quantum processing and load data from SAVE_PATH
np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)

**2.2 Loading MNIST Dataset :**

**A) Description :** MNIST is a large database that comprises 28 × 28 gray images of handwritten digits from 0 to 9. It is usually use to train and test image processing systems.

**B) Data Preparation :** In this example, we are going to use only a subsection of the MNIST dataset. We are doing this to reduce the computation time taken to complete of our code. We also reduce the number of samples for both training and testing.

**C) Normalization :** This normalizes the pixel values for the images to lie within the range of [0, 1]. This step is crucial for the implementation of the neural network.

**D) Adding a channel :** It adds an extra dimension to represent the convolution channels to each of the images because the convolutional operation requires the 3D space. 

In [ ]:
mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

### 3. Quantum Circuit as a Convolution Kernel

Now we are defining the quantum circuit which shall play the role of convolution kernel. The quantum circuit is called for small image patches of the input image and the expectation values thus returned are used as the feature input for further layers.

**3.1 Device Initialization :**

**A) Device :** We use PennyLane's default.qubit simulator. This is a quantum device simulator with a 4-qubit quantum device.

**B) Random Parameters :** Parameters for the quantum circuit are initialized with random values. It is these parameters which are responsible for effecting the dynamics of the quantum gates.



**3.2 Quantum Circuit Definition**

**A) Encoding :** The quantum processing circuit encodes the 2x2 pixel region from the input image using parameterised rotation gates. In particular, we use the RYRY gate with angles scaled in increments of ππ angles.

**B) Random Layers :** A set of random quantum layers are appended, sampled from the distribution defined by the variational parameters at initialization. This introduces the randomness and complexity in the kernel.

**C) Measurement :** The expectation values of the Pauli-Z operator for each qubit is measured. These measurements are the quantum features taken from the input region.


In [ ]:
dev = qml.device("default.qubit", wires=4)
# Random circuit parameters
rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, 4))

@qml.qnode(dev)
def circuit(phi):
    # Encoding of 4 classical input values
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(4)))

    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

**3.3 Convolution Scheme**

This procedure defines the process of applying quantum convolution to the input image.

**Steps :**

**A) Input Division :** The input image is divided into non-overlapping 2x2 pixel units.

**B) Quantum Procedure :** The each pixel unit undergo the quantum procedure defined above so that the expectation values are returned.

**C) Channel Assignation :** Expectation values are assigned to the four channels of the single output pixel.This is a convolution of 2x2 stride 2.

**D) Output Builder :** The whole image goes through this process to produce a downsampled output image of multiple channels.


In [ ]:
def quanv(image):
    """Convolves the input image with many applications of the same quantum circuit."""
    out = np.zeros((14, 14, 4))

    # Loop over the coordinates of the top-left pixel of 2X2 squares
    for j in range(0, 28, 2):
        for k in range(0, 28, 2):
            # Process a squared 2x2 region of the image with a quantum circuit
            q_results = circuit(
                [
                    image[j, k, 0],
                    image[j, k + 1, 0],
                    image[j + 1, k, 0],
                    image[j + 1, k + 1, 0]
                ]
            )
            # Assign expectation values to different channels of the output pixel (j/2, k/2)
            for c in range(4):
                out[j // 2, k // 2, c] = q_results[c]
    return out

**3.4 Quantum pre-processing of the dataset**

Since quantum convolution is actually a fixed, non-trainable layer, we apply it in the pre-processing to the entire dataset. Thus, during the actual training, it maintains reasonable computational overhead.

**A) Train Images :** The quantum convolution function is applied to each image of the training.
Set and storing the outputs.

**B) Test Images :** Process of test set

**C) Output Saving :** Output is saved, so it can be used in subsequent operations


In [ ]:
if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    # Save pre-processed images
    np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images.npy", q_test_images)


# Load pre-processed images
q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
q_test_images = np.load(SAVE_PATH + "q_test_images.npy")

**3.5 Quantum Convolution Effect Visualization**

In this quantum convolution, we visualize the effect of the Quantum Convolution on the input images.

**Visualization:**

**A) Input Image :** Sample input images from the training dataset is taken.

**B) Quantum Features :** For each input image, we draw the output of each channel when that input is processed into the Quantum conv layer. Each channel illustrates a different quantum feature which is learned from the input.


In [ ]:
n_samples = 4
n_channels = 4
fig, axes = plt.subplots(1 + n_channels, n_samples, figsize=(10, 10))
for k in range(n_samples):
    axes[0, 0].set_ylabel("Input")
    if k != 0:
        axes[0, k].yaxis.set_visible(False)
    axes[0, k].imshow(train_images[k, :, :, 0], cmap="gray")

    # Plot all output channels
    for c in range(n_channels):
        axes[c + 1, 0].set_ylabel("Output [ch. {}]".format(c))
        if k != 0:
            axes[c, k].yaxis.set_visible(False)
        axes[c + 1, k].imshow(q_train_images[k, :, :, c], cmap="gray")

plt.tight_layout()
plt.show()

### 4. Hybrid Quantum-Classical Model

Here we define a simple classical neural network which we will train on the quantum-processed images. This hybrid approach combines quantum feature extraction with classical training.

**4.1 Model Definiton :**

**A) Input Layer :** No input layer

**B) Dense Layer :** 10 neurons, Softmax activation. The dense layer or classification layer.

**C) Compile :** The model is then compiled with the Adam optimizer and sparse categorical cross-entropy loss, as this is a multi-class classification problem.


In [ ]:
def MyModel():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

**4.2 Training :**

We first initialize an instance of the model, then we train and validate it with the dataset that has been already pre-processed by a quantum convolution.

In [ ]:
# Create an instance of your model (unindented)
q_model = MyModel()  

q_history = q_model.fit(
    q_train_images,
    train_labels,
    validation_data=(q_test_images, test_labels),
    batch_size=4,
    epochs=n_epochs,
    verbose=2,
)

In order to compare the results achievable with and without the quantum convolution layer, we initialize also a “classical” instance of the model that will be directly trained and validated with the raw MNIST images (i.e., without quantum pre-processing).

In [ ]:
c_model = MyModel()

c_history = c_model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size=4,
    epochs=n_epochs,
    verbose=2,
)

**4.3 Results**

We can finally plot the test accuracy and the test loss with respect to the number of training epochs.

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("seaborn")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 9))

ax1.plot(q_history.history["val_accuracy"], "-ob", label="With quantum layer")
ax1.plot(c_history.history["val_accuracy"], "-og", label="Without quantum layer")
ax1.set_ylabel("Accuracy")
ax1.set_ylim([0, 1])
ax1.set_xlabel("Epoch")
ax1.legend()

ax2.plot(q_history.history["val_loss"], "-ob", label="With quantum layer")
ax2.plot(c_history.history["val_loss"], "-og", label="Without quantum layer")
ax2.set_ylabel("Loss")
ax2.set_ylim(top=2.5)
ax2.set_xlabel("Epoch")
ax2.legend()
plt.tight_layout()
plt.show()